## Import Modules

In [5]:
import pandas as pd
from glob import glob
import IPython.display as ipd

In [7]:
import numpy as np
import random
import os
import torch

from scipy.io import wavfile
import noisereduce as nr

/root/rokafnet/.venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Fix Seed

In [8]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
seed_everything()

## Config

In [9]:
CFG = {
    'model': 'openai/whisper-tiny',
    'sr': 16000,
    'noise_file_path': '../files/noise.wav'
}

## Read Files

In [10]:
TRAIN_PATH = '/mnt/elice/dataset/train/'
TEST_PATH = '/mnt/elice/dataset/test/'

In [11]:
df = pd.read_csv(f'{TRAIN_PATH}/texts.csv', index_col=False)
submission = pd.read_csv(f'sample_submission.csv', index_col=False)

## Data Cleaning

In [12]:
import re

def clean_text(text, remove_space=True):
    text = re.sub(r'[!"#$%&\'()*+,-./:;<=>?@\[\]^_\`{|}~\\\\]','', text)
    if remove_space:
        text = ''.join(text.split())
    return text

In [13]:
# Label Cleaning (remove punctuations)
df['text'] = df['text'].apply(lambda x: clean_text(x, False))

# remove outlier data
df = df[df['filenames'] != 'audio_5497.wav']

if not os.path.exists('preprocess'):
    os.mkdir('preprocess')

df.to_csv('preprocess/clean_df.csv', index=False)

## Split long/short dataframe

In [14]:
import librosa
import pandas as pd
from tqdm.auto import tqdm

def split_dataframe(df, df_name, is_train=True):
    df_long = []
    df_short = []

    for idx, row in tqdm(df.iterrows()):
        if is_train:
            path = TRAIN_PATH + row['filenames']
        else:
            path = row['path']
        wav, fs = librosa.load(path)
        length = len(wav)/fs

        if length >= 30:
            df_long.append(row)
        else:
            df_short.append(row)

    df_long = pd.DataFrame(df_long, columns=df.columns)
    df_short = pd.DataFrame(df_short, columns=df.columns)

    df_long.to_csv(f'preprocess/long_{df_name}.csv', index=False)
    df_short.to_csv(f'preprocess/short_{df_name}.csv', index=False)

In [15]:
split_dataframe(df, 'df')
split_dataframe(submission, 'test', False)

13999it [00:48, 287.87it/s]
6000it [00:23, 255.25it/s]


## Data Preprocess & Train Dataset

In [17]:
from transformers import WhisperTokenizer,  WhisperFeatureExtractor
from transformers import WhisperProcessor
from scipy.io import wavfile

# load feature extractor and tokenizer
feature_extractor = WhisperFeatureExtractor.from_pretrained(CFG['model'])
tokenizer = WhisperTokenizer.from_pretrained(CFG['model'], language="Korean", task="transcribe")

_, noise_array = wavfile.read(CFG["noise_file_path"])

In [18]:
def prepare_dataset(batch):
    audio = batch['audio']
    reduced_noise_audio = nr.reduce_noise(y=audio['array'], sr=CFG['sr'], y_noise = noise_array)

    # raw form(reduced_noise_audio) -> log-Mel spectrogram
    batch['input_features'] = feature_extractor(reduced_noise_audio, sampling_rate=audio['sampling_rate']).input_features[0]
    
    # target text -> label ids(by tokenizer)
    batch['labels'] = tokenizer(batch['transcripts']).input_ids

    return batch

In [19]:
from datasets import Dataset, DatasetDict
from datasets import Audio

def create_train_datasets(df, dir_name='dataset'):
    # create dataset from csv
    ds = Dataset.from_dict({"audio": [f'{TRAIN_PATH}/{file_path}' for file_path in df["filenames"]],
                        "transcripts": [text for text in df["text"]]}).cast_column("audio", Audio(sampling_rate=CFG['sr']))

    # train/valid split
    train_valid = ds.train_test_split(test_size=0.2)
    train_valid_dataset = DatasetDict({
        "train": train_valid["train"],
        "valid": train_valid["test"]})
    
    train_valid_dataset = train_valid_dataset.map(prepare_dataset, remove_columns = train_valid_dataset.column_names['train'], num_proc=4)

    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
        
    train_valid_dataset.save_to_disk(dir_name)

In [20]:
# create_train_datasets(df)

# create long/short train dataset from csv files
# long_df = pd.read_csv('preprocess/long_df.csv', index_col=False)
short_df = pd.read_csv('preprocess/short_df.csv', index_col=False)

# create_train_datasets(long_df, dir_name='dataset_long')
create_train_datasets(short_df, dir_name='dataset_short')

Saving the dataset (6/6 shards): 100%|██████████| 2772/2772 [00:03<00:00, 700.25 examples/s]


## Test Dataset

In [21]:
def prepare_test_dataset(batch):
    audio = batch['audio']
    reduced_noise_audio = nr.reduce_noise(y=audio['array'], sr=CFG['sr'], y_noise = noise_array)

    # raw form(reduced_noise_audio) -> log-Mel spectrogram
    batch['input_features'] = feature_extractor(reduced_noise_audio, sampling_rate=audio['sampling_rate']).input_features[0]

    return batch

In [22]:
from datasets import Dataset, DatasetDict
from datasets import Audio

def create_test_dataset(df, dir_name='dataset_test'):
    # create dataset from csv
    test_dataset = Dataset.from_dict({"audio": [file_path for file_path in df["path"]]})
    test_dataset = test_dataset.cast_column("audio", Audio(sampling_rate=CFG['sr']))
    sampling_rate = test_dataset.features['audio'].sampling_rate

    # test data preprocess
    test_dataset = test_dataset.map(prepare_test_dataset, remove_columns = test_dataset.column_names, num_proc=4)

    # save test dataset
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
        
    test_dataset.save_to_disk(dir_name)

In [23]:
create_test_dataset(submission)

# create long/short test dataset from csv files
# long_test = pd.read_csv('preprocess/long_test.csv', index_col=False)
# short_test = pd.read_csv('preprocess/short_test.csv', index_col=False)

# create_test_dataset(long_test, dir_name='dataset_long_test')
# create_test_dataset(short_test, dir_name='dataset_short_test')

Saving the dataset (12/12 shards): 100%|██████████| 6000/6000 [00:02<00:00, 2208.39 examples/s]
